# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: LoRA - LOW-RANK ADAPTATION OF LARGE LANGUAGE MODELS
* Model: cardiffnlp/twitter-roberta-base-sentiment-latest
* Evaluation approach: Transformer
* Fine-tuning dataset: Twitter dataset


## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

### Prior to fine-tuning. 
Here choosen "cardiffnlp/twitter-roberta-base-sentiment-latest" Modle to evaluate and understand, Brief about the model - This is a RoBERTa-base model trained on ~124M tweets from January 2018 to December 2021, and finetuned for sentiment analysis with the TweetEval benchmark. The original Twitter-based RoBERTa model can be found here and the original reference paper is TweetEval. This model is suitable for English.


In [2]:
!pip install transformers
!pip install peft
!pip install datasets
!pip install pandas
!pip install numpy
!pip install scikit-learn
!pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 59.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 40.3 MB/s eta 0:00:00
Defaulting to user installation because normal site-packages is not writeable


In [3]:
from transformers import AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
from transformers import pipeline
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EvalPrediction
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from peft import LoraConfig, PeftModelForSequenceClassification, TaskType, AutoPeftModelForSequenceClassification
import torch
import tqdm

#model_path = "cardiffnlp/twitter-roberta-base-sentiment-latest"
#sentiment_task = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path)
#sentiment_task("Climate is becoming hotter every year!")

def preprocess(text):
    new_text=[]
    for t in text.split(" "):
        t = '@user' if t.startswith("@") and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)
        
print(preprocess("Climate is becoming hotter every year!"))

/opt/conda/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
Climate is becoming hotter every year!


In [4]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

text = "Climate change is a imminent threat!"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


1) negative 0.6982
2) neutral 0.261
3) positive 0.0408


In [5]:
import pandas as pd
from datasets import load_dataset
dataset = load_dataset("tweet_eval", "sentiment")
#for i in range(10):
#    print(dataset['train'][i])
    
unique_sentiment = pd.unique(dataset['train']['label'])

print(unique_sentiment[::-1])
df = pd.DataFrame(dataset['train'])
df.head(10)

Generating train split:   0%|          | 0/45615 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/12284 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

/tmp/ipykernel_13/740065493.py:7: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  unique_sentiment = pd.unique(dataset['train']['label'])


[0 1 2]


,text,label
0,"""QT @user In the original draft of the 7th boo...",2
1,"""Ben Smith / Smith (concussion) remains out of...",1
2,Sorry bout the stream last night I crashed out...,1
3,Chase Headley's RBI double in the 8th inning o...,1
4,@user Alciato: Bee will invest 150 million in ...,2
5,@user LIT MY MUM 'Kerry the louboutins I wonde...,2
6,"""\"""""""" SOUL TRAIN\"""""""" OCT 27 HALLOWEEN SPECIA...",2
7,So disappointed in wwe summerslam! I want to s...,0
8,"""This is the last Sunday w/o football .....,NF...",2
9,@user @user CENA & AJ sitting in a tree K-I-S-...,1


In [6]:

#df['label'] = df['sentiment'].map(unique_sentiment)

train_df, val_df = train_test_split(df, test_size=0.1, stratify=df['label'])

# Convert the dataframes into Hugging Face datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Define the tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL)
tokenizer.pad_token = tokenizer.eos_token

# Tokenize and convert
def tokenize_and_encode(examples):
    tokenized_inputs = tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)
    tokenized_inputs['labels'] = examples['label']
    return tokenized_inputs

train_dataset = train_dataset.map(tokenize_and_encode, batched=True)
val_dataset = val_dataset.map(tokenize_and_encode, batched=True)

Map:   0%|          | 0/41053 [00:00<?, ? examples/s]

Map:   0%|          | 0/4562 [00:00<?, ? examples/s]

In [ ]:
#train_dataset[0]

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [7]:
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=4,
    lora_alpha=16,
    lora_dropout=0.1
)

# Load the pre-trained model
model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=len(unique_sentiment))
model.config.pad_token_id = model.config.eos_token_id

peft_model = PeftModelForSequenceClassification(model, peft_config)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
# Print
peft_model.print_trainable_parameters()



Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


trainable params: 1,333,254 || all params: 125,388,294 || trainable%: 1.0633002152497584


In [9]:
import numpy as np
from transformers import DataCollatorWithPadding, AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EvalPrediction



def compute_metrics(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average='weighted')
    return {"accuracy": accuracy_score(p.label_ids, preds), "f1": f1, "precision": precision, "recall": recall}

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir='./logs',
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_steps=100,
    warmup_ratio=0.1,
)

# Initialize the Trainer with compute_metrics
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Start training
trainer.train()

# Evaluate
evaluation_results = trainer.evaluate()
print("Evaluation Results:", evaluation_results)

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.488600,0.534537,0.771591,0.771722,0.772427,0.771591


Evaluation Results: {'eval_loss': 0.5345368385314941, 'eval_accuracy': 0.7715914072775099, 'eval_f1': 0.7717215728186169, 'eval_precision': 0.7724268608349637, 'eval_recall': 0.7715914072775099, 'eval_runtime': 147.047, 'eval_samples_per_second': 31.024, 'eval_steps_per_second': 1.945, 'epoch': 1.0}


In [14]:
peft_model.save_pretrained('model/peft_model')

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [15]:
inference_model = AutoPeftModelForSequenceClassification.from_pretrained(
    "model/peft_model",
    num_labels=len(unique_sentiment)
)
inference_model.config.pad_token_id = inference_model.config.eos_token_id

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
trainer = Trainer(
    model=inference_model,
    args=training_args,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Evaluate the model
evaluation_results = trainer.evaluate()
print("Evaluation Results:", evaluation_results)

Evaluation Results: {'eval_loss': 0.5345368385314941, 'eval_accuracy': 0.7715914072775099, 'eval_f1': 0.7717215728186169, 'eval_precision': 0.7724268608349637, 'eval_recall': 0.7715914072775099, 'eval_runtime': 148.323, 'eval_samples_per_second': 30.757, 'eval_steps_per_second': 1.928}


In [17]:
def predict(sentence: str) -> str:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    inference_model.to(device)

    # Prepare the input text
    inputs = tokenizer(sentence, return_tensors="pt").to(device)

    # Get predictions
    with torch.no_grad():
        outputs = inference_model(**inputs)
        logits = outputs.logits

    probabilities = torch.nn.functional.softmax(logits, dim=1)
    print(f" probabilities -- {probabilities}")
    predicted_class_id = probabilities.argmax().item()
    print(f" probabilities -- {predicted_class_id}")
    predicted_label = predicted_class_id

    return predicted_label

# Example usage
#1) negative 0.6982
#2) neutral 0.261
#3) positive 0.0408
sentiment = {2: 'positive', 1: 'negative', 0: 'neutral'}
sentence = "Giant asteroid heading towards earth"
predicted_label = predict(sentence)
print(f"Sentence: '{sentence}'\nPredicted label: {sentiment[predicted_label]}")

 probabilities -- tensor([[0.1968, 0.5190, 0.2842]])
 probabilities -- 1
Sentence: 'Giant asteroid heading towards earth'
Predicted label: negative


In [18]:
#Example 2
sentence = "After the rapid adoption of EV pollution seems to be coming under control"
predicted_label = predict(sentence)
print(f"Sentence: '{sentence}'\nPredicted label: {sentiment[predicted_label]}")

 probabilities -- tensor([[0.0436, 0.6188, 0.3376]])
 probabilities -- 1
Sentence: 'After the rapid adoption of EV pollution seems to be coming under control'
Predicted label: negative


In [19]:
#Example 3
sentence = "All are happy"
predicted_label = predict(sentence)
print(f"Sentence: '{sentence}'\nPredicted label: {sentiment[predicted_label]}")

 probabilities -- tensor([[0.0087, 0.0473, 0.9441]])
 probabilities -- 2
Sentence: 'All are happy'
Predicted label: positive


In [20]:
#Example 4
sentence = "Not sure how the future is looking with this economy!"
predicted_label = predict(sentence)
print(f"Sentence: '{sentence}'\nPredicted label: {sentiment[predicted_label]}")

 probabilities -- tensor([[0.8169, 0.1654, 0.0177]])
 probabilities -- 0
Sentence: 'Not sure how the future is looking with this economy!'
Predicted label: neutral


In [21]:
#Example 5
sentence = "average life expectancy has generally increased over the past century in many parts of the world, thanks to advancements in medical technology, improved healthcare"
predicted_label = predict(sentence)
print(f"Sentence: '{sentence}'\nPredicted label: {sentiment[predicted_label]}")

 probabilities -- tensor([[0.0045, 0.1081, 0.8873]])
 probabilities -- 2
Sentence: 'average life expectancy has generally increased over the past century in many parts of the world, thanks to advancements in medical technology, improved healthcare'
Predicted label: positive


In [22]:
#Example 6
sentence = "All are happy Or may be not. "
predicted_label = predict(sentence)
print(f"Sentence: '{sentence}'\nPredicted label: {sentiment[predicted_label]}")

 probabilities -- tensor([[0.0291, 0.2961, 0.6748]])
 probabilities -- 2
Sentence: 'All are happy Or may be not. '
Predicted label: positive


In [24]:
#Example 7
sentence = "And I think to myself what a wonderful world Yes I think to myself what a wonderful world"
predicted_label = predict(sentence)
print(f"Sentence: '{sentence}'\nPredicted label: {sentiment[predicted_label]}")

 probabilities -- tensor([[0.0023, 0.0161, 0.9816]])
 probabilities -- 2
Sentence: 'And I think to myself what a wonderful world Yes I think to myself what a wonderful world'
Predicted label: positive
